In [7]:
import pandas as pd
import re

from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By

cols = ['Nama', 'Harga Diskon', 'Harga Awal', 'Diskon', 'Rating', 'Terjual', 'Kota/Kab', 'Feedback', 'COD', 'No Ongkir']
df = pd.DataFrame(columns = cols)

service = Service("C:\Program Files\edgedriver_win64")
driver = webdriver.Edge(service = service)

url = 'https://www.bukalapak.com/products?search%5Bkeywords%5D=lighting'

driver.get(url)
driver.implicitly_wait(5)

In [8]:
def is_float(string):
    try:
        float(string)
        return True
    
    except ValueError:
        return False

In [36]:
res = driver.find_elements(By.XPATH, "//div[@class='bl-flex-container direction-column']")
list_items = res[1].text.split('\n')[3:]

idx = list_items.index('Pencarian Terkait')
list_items = list_items[:idx]

ValueError: 'Pencarian Terkait' is not in list

In [ ]:
i = 0
while i < len(list_items):
    nama = ''
    harga_diskon = ''
    harga_awal = ''
    diskon = ''
    rating = ''
    terjual = ''
    kota_kab = ''
    feedback = ''
    cod = 0
    no_ongkir = 0

    count = i
    j = 0
    while (j < 11) and (count + j < len(list_items)):

        if (j == 0) or (len(list_items[count + j]) >= 25):
            if j == 0:
                nama = list_items[count + j]

            else:
                j = 11

        elif j == 1:
            pass

        elif j == 2:
            harga_diskon = list_items[count + j]

        elif list_items[count + j].endswith('%'):
            diskon = list_items[count + j][:len(list_items[count + j]) - 1]
            harga_awal = list_items[count + j + 1]
            
            j += 1

        elif is_float(list_items[count + j]):
            if float(list_items[count + j]) <= 5:
                rating = list_items[count + j]

        elif 'Terjual' in list_items[count + j]:
            terjual = list_items[count + j].split(' ')[1]

        elif 'Feedback' in list_items[count + j]:
            feedback = list_items[count + j].split(' ')[0]
            kota_kab = list_items[count + j - 1]

        elif list_items[count + j] == 'Bayar di Tempat (COD)':
            cod = 1

        elif list_items[count + j] == 'No Ongkir':
            no_ongkir = 1

        j += 1

    if harga_awal == '':
        harga_awal = harga_diskon
        harga_diskon = ''

    temp = [nama, harga_diskon, harga_awal, diskon, rating, terjual, kota_kab, feedback, cod, no_ongkir]

    i += 1
    i += len([_ for _ in temp if _ not in ['', 0]])

    df.loc[len(df)] = temp

In [35]:
df

,Nama,Harga Diskon,Harga Awal,Diskon,Rating,Terjual,Kota/Kab,Feedback,COD,No Ongkir
0,VYATTA BULLET LIGHTNING USB CABLE - GARANSI 12...,67.150,Rp79.000,15,4.9,213,Jakarta Timur,34rb,0,1
1,Oraimo Charger Handphone Firefly 3 Quick Charg...,50.915,Rp89.000,33,5,1,Jakarta Pusat,4rb,1,1
2,Softbox Flash 50cm x 70cm Lighting Studio Stro...,107.300,Rp145.000,26,5,2,Kab. Tangerang,16rb,1,1
3,Portable Light Stand Tripod 200cm for Studio L...,45.800,Rp188.000,76,,,Jakarta Barat,11rb,1,1
4,Stand Tripod 180cm for Studio Lighting W803,45.600,Rp195.000,77,5,1,Jakarta Barat,15rb,0,1
...,...,...,...,...,...,...,...,...,...,...
449,Kabel data Apple Lighting Cable Iphone 2.0A JO...,18.000,Rp70.000,74,5,86,Jakarta Utara,33rb,1,1
450,Kabel Data Cable Data 3 In 1 MIcro USB Type-C ...,34.900,Rp70.000,50,4.9,558,Jakarta Utara,33rb,1,1
451,Kabel Data Cable Data 90 Degree Apple Lighting...,24.900,Rp60.000,59,4.9,259,Jakarta Utara,33rb,1,1
452,Kabel data Cable data Apple Lighting Cable Iph...,14.900,Rp25.000,40,4.9,701,Jakarta Utara,33rb,1,1


In [26]:
import pandas as pd
import re

df = pd.read_csv('lighting.csv')
df

,Nama,Harga Diskon,Harga Awal,Diskon,Rating,Terjual,Kota/Kab,Feedback
0,VYATTA BULLET LIGHTNING USB CABLE - GARANSI 12...,67.150,Rp79.000,213.0,4.9,213.0,Jakarta Timur,34rb
1,Oraimo Charger Handphone Firefly 3 Quick Charg...,50.915,Rp89.000,1.0,5.0,1.0,Jakarta Pusat,4rb
2,Softbox Flash 50cm x 70cm Lighting Studio Stro...,107.300,Rp145.000,2.0,5.0,2.0,Kab. Tangerang,16rb
3,Portable Light Stand Tripod 200cm for Studio L...,45.800,Rp188.000,NaN,NaN,NaN,Jakarta Barat,11rb
4,Stand Tripod 180cm for Studio Lighting W803,45.600,Rp195.000,1.0,5.0,1.0,Jakarta Barat,15rb
...,...,...,...,...,...,...,...,...
359,Kabel data Apple Lighting Cable Iphone 2.0A JO...,18.000,Rp70.000,86.0,5.0,86.0,Jakarta Utara,33rb
360,Kabel Data Cable Data 3 In 1 MIcro USB Type-C ...,34.900,Rp70.000,558.0,4.9,558.0,Jakarta Utara,33rb
361,Kabel Data Cable Data 90 Degree Apple Lighting...,24.900,Rp60.000,259.0,4.9,259.0,Jakarta Utara,33rb
362,Kabel data Cable data Apple Lighting Cable Iph...,14.900,Rp25.000,701.0,4.9,701.0,Jakarta Utara,33rb


In [47]:
df = df.drop_duplicates()
df['Harga Awal'] = df['Harga Awal'].apply(lambda x: re.sub(r'[^0-9.]', '', x) if type(x) == str else x)
df['Harga Diskon'] = df['Harga Diskon'].apply(lambda x: re.sub(r'[^0-9.]', '', x) if type(x) == str else x)
df['Harga Diskon'] = df['Harga Diskon'].fillna(df['Harga Awal'])
df = df.dropna(subset = ['Harga Awal', 'Harga Diskon'])
df = df.loc[~(df['Harga Awal'] == '')]
df['Harga Awal'] = df['Harga Awal'].astype(float)
df['Harga Diskon'] = df['Harga Diskon'].astype(float)

df['Diskon'] = round((df['Harga Awal'] - df['Harga Diskon']) / df['Harga Awal'] * 100, 1)

df.to_csv('lighting.csv', index = False)

df

,Nama,Harga Diskon,Harga Awal,Diskon,Rating,Terjual,Kota/Kab,Feedback
0,VYATTA BULLET LIGHTNING USB CABLE - GARANSI 12...,67150.0,79000.0,15.0,4.9,213.0,Jakarta Timur,34rb
1,Oraimo Charger Handphone Firefly 3 Quick Charg...,50915.0,89000.0,42.8,5.0,1.0,Jakarta Pusat,4rb
2,Softbox Flash 50cm x 70cm Lighting Studio Stro...,107300.0,145000.0,26.0,5.0,2.0,Kab. Tangerang,16rb
3,Portable Light Stand Tripod 200cm for Studio L...,45800.0,188000.0,75.6,NaN,NaN,Jakarta Barat,11rb
4,Stand Tripod 180cm for Studio Lighting W803,45600.0,195000.0,76.6,5.0,1.0,Jakarta Barat,15rb
...,...,...,...,...,...,...,...,...
359,Kabel data Apple Lighting Cable Iphone 2.0A JO...,18000.0,70000.0,74.3,5.0,86.0,Jakarta Utara,33rb
360,Kabel Data Cable Data 3 In 1 MIcro USB Type-C ...,34900.0,70000.0,50.1,4.9,558.0,Jakarta Utara,33rb
361,Kabel Data Cable Data 90 Degree Apple Lighting...,24900.0,60000.0,58.5,4.9,259.0,Jakarta Utara,33rb
362,Kabel data Cable data Apple Lighting Cable Iph...,14900.0,25000.0,40.4,4.9,701.0,Jakarta Utara,33rb
